In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 分布式顶点 AI 超参数调整

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/training/distributed_hyperparameter_tuning.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在 Colab 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Ftraining%2Fdistributed_hyperparameter_tuning.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在 Colab Enterprise 中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/training/distributed_hyperparameter_tuning.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在 Workbench 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/training/distributed_hyperparameter_tuning.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在 GitHub 上查看
    </a>
  </td>
</table>

## 概述

本笔记本演示如何使用Vertex AI Training来运行一个超参数调整作业，以发现ML模型的最佳超参数值。为了加快训练过程，使用了`tf.distribute`模块中的`MirroredStrategy`来在单台机器上的多个GPU上分发训练。

了解更多关于[Vertex AI超参数调整](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview)。

### 目标

在这个笔记本中，您将在一个Docker容器中从Python脚本创建一个自定义训练模型。您将学习如何修改训练应用程序代码以进行超参数调整，并使用Python SDK提交Vertex AI超参数调整作业。

本教程使用以下谷歌云ML服务：

- `Vertex AI训练`
- `Vertex AI超参数调整`

执行的步骤包括：

- 使用Python包进行训练。
- 在超参数调整时报告准确度。
- 使用GCSFuse将模型工件保存到Cloud Storage。

### 数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets)的“马或人类数据集”。训练模型可以预测图像是马还是人的。

### 成本

本教程使用Google Cloud的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI的定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage的定价](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)基于您的预期使用情况生成成本估算。

开始吧

安装Vertex AI SDK for Python和其他所需的包

In [ ]:
! pip3 install --upgrade google-cloud-aiplatform -q

### 重新启动运行时（仅适用于Colab）

要使用新安装的软件包，您必须重新启动Google Colab上的运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ 内核将重新启动。等待它完成后再继续下一步。⚠️</b>
</div>

### 在Colab上验证您的笔记本环境

在Google Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置Google Cloud项目信息并初始化Python的Vertex AI SDK

要开始使用Vertex AI，您必须拥有一个现有的Google Cloud项目并[启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

创建一个云存储桶

创建一个存储桶来存储中间产品，如数据集。

In [ ]:
BUCKET_URI = f"gs://dist-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶不存在时：运行以下单元格来创建您的云存储桶。

In [ ]:
! gsutil mb -l {LOCATION} {BUCKET_URI}

### 导入库

In [ ]:
from google.cloud import aiplatform
from google.cloud.aiplatform import hyperparameter_tuning as hpt

编写Dockerfile

容器化代码的第一步是创建一个Dockerfile。在Dockerfile中，您将包含运行映像所需的所有命令，例如安装必要的库和设置训练代码的入口点。

此Dockerfile使用了Deep Learning Container TensorFlow Enterprise 2.5 GPU Docker映像。Google Cloud上的Deep Learning Containers预装了许多常见的ML和数据科学框架。下载该映像后，此Dockerfile安装了[CloudML Hypertune](https://github.com/GoogleCloudPlatform/cloudml-hypertune)库，并设置了训练代码的入口点。

In [ ]:
%%writefile Dockerfile

FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-6
WORKDIR /

# Installs hypertune library
RUN pip install cloudml-hypertune

# Copies the trainer code to the docker image.
COPY trainer /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

### 创建训练应用程序代码

接下来，你需要创建一个trainer目录，其中包含一个名为`task.py`的脚本，用来存储你的训练应用程序的代码。

In [ ]:
# Create trainer directory

! mkdir trainer

在下一个单元格中，您将编写训练脚本`task.py`的内容。这个文件从TensorFlow数据集中下载_horses or humans_数据集，并使用`tf.distribute`模块中的`MirroredStrategy`训练一个`tf.keras`功能模型。

有一些专门用于使用超参数调整服务的组件：

* 脚本导入了`hypertune`库。请注意，Dockerfile中包含了pip安装hypertune库的说明。
* 函数`get_args()`为您希望调整的每个超参数定义了一个命令行参数。在这个例子中，将调整的超参数是学习率、优化器中的动量值和模型中最后一个隐藏层中的单元数。传递给这些参数的值然后用于设置代码中相应的超参数。
* 在`main()`函数的末尾，使用`hypertune`库定义要优化的指标。在这个例子中，将优化的指标是验证准确率。这一指标传递给`HyperTune`的实例。

In [ ]:
%%writefile trainer/task.py

import argparse
import hypertune
import tensorflow as tf
import tensorflow_datasets as tfds

def get_args():
  """Parses args. Must include all hyperparameters you want to tune."""

  parser = argparse.ArgumentParser()
  parser.add_argument(
      '--learning_rate', required=True, type=float, help='learning rate')
  parser.add_argument(
      '--momentum', required=True, type=float, help='SGD momentum value')
  parser.add_argument(
      '--units',
      required=True,
      type=int,
      help='number of units in last hidden layer')
  parser.add_argument(
      '--epochs',
      required=False,
      type=int,
      default=10,
      help='number of training epochs')
  args = parser.parse_args()
  return args


def preprocess_data(image, label):
  """Resizes and scales images."""

  image = tf.image.resize(image, (150, 150))
  return tf.cast(image, tf.float32) / 255., label


def create_dataset(batch_size):
  """Loads Horses Or Humans dataset and preprocesses data."""

  data, info = tfds.load(
      name='horses_or_humans', as_supervised=True, with_info=True)

  # Create train dataset
  train_data = data['train'].map(preprocess_data)
  train_data = train_data.shuffle(1000)
  train_data = train_data.batch(batch_size)

  # Create validation dataset
  validation_data = data['test'].map(preprocess_data)
  validation_data = validation_data.batch(64)

  return train_data, validation_data


def create_model(units, learning_rate, momentum):
  """Defines and compiles model."""

  inputs = tf.keras.Input(shape=(150, 150, 3))
  x = tf.keras.layers.Conv2D(16, (3, 3), activation='relu')(inputs)
  x = tf.keras.layers.MaxPooling2D((2, 2))(x)
  x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(x)
  x = tf.keras.layers.MaxPooling2D((2, 2))(x)
  x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
  x = tf.keras.layers.MaxPooling2D((2, 2))(x)
  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dense(units, activation='relu')(x)
  outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
  model = tf.keras.Model(inputs, outputs)
  model.compile(
      loss='binary_crossentropy',
      optimizer=tf.keras.optimizers.SGD(
          learning_rate=learning_rate, momentum=momentum),
      metrics=['accuracy'])
  return model


def main():
  args = get_args()

  # Create Strategy
  strategy = tf.distribute.MirroredStrategy()

  # Scale batch size
  GLOBAL_BATCH_SIZE = 64 * strategy.num_replicas_in_sync  
  train_data, validation_data = create_dataset(GLOBAL_BATCH_SIZE)

  # Wrap model variables within scope
  with strategy.scope():
    model = create_model(args.units, args.learning_rate, args.momentum)

  # Train model
  history = model.fit(
      train_data, epochs=args.epochs, validation_data=validation_data)

  # Define Metric
  hp_metric = history.history['val_accuracy'][-1]

  hpt = hypertune.HyperTune()
  hpt.report_hyperparameter_tuning_metric(
      hyperparameter_metric_tag='accuracy',
      metric_value=hp_metric,
      global_step=args.epochs)


if __name__ == '__main__':
  main()

### 构建容器

在接下来的单元格中，您将构建容器并将其推送到Google容器注册表。

In [ ]:
# Set the IMAGE_URI
IMAGE_URI = f"gcr.io/{PROJECT_ID}/horse-human:hypertune"

In [ ]:
# Build the docker image
! docker build -f Dockerfile -t $IMAGE_URI ./

In [ ]:
# Push it to Google Container Registry:
! docker push $IMAGE_URI

### 在Vertex AI上创建和运行超参数调优任务

一旦您的容器被推送到Google容器注册表，您可以使用Vertex SDK来创建和运行超参数调优任务。

您定义以下规格：
* `worker_pool_specs`：指定机器类型和Docker镜像的字典。这个示例定义了一个带有一个`n1-standard-4`机器和两个`NVIDIA_TESLA_T4` GPU的单节点集群。
* `parameter_spec`：指定要优化的参数的字典。字典键是分配给您训练应用程序代码中每个超参数的命令行参数的字符串，字典值是参数规范。参数规范包括超参数的类型、最小/最大值和比例。
* `metric_spec`：指定要优化的指标的字典。字典键是您在训练应用程序代码中设置的`hyperparameter_metric_tag`，值是优化目标。

In [ ]:
worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": "n1-standard-4",
            "accelerator_type": "NVIDIA_TESLA_T4",
            "accelerator_count": 2,
        },
        "replica_count": 1,
        "container_spec": {"image_uri": IMAGE_URI},
    }
]

metric_spec = {"accuracy": "maximize"}

parameter_spec = {
    "learning_rate": hpt.DoubleParameterSpec(min=0.001, max=1, scale="log"),
    "momentum": hpt.DoubleParameterSpec(min=0, max=1, scale="linear"),
    "units": hpt.DiscreteParameterSpec(values=[64, 128, 512], scale=None),
}

创建一个`CustomJob`。

In [ ]:
# Create a CustomJob

JOB_NAME = "horses-humans-hyperparam-job"

my_custom_job = aiplatform.CustomJob(
    display_name=JOB_NAME,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=BUCKET_URI,
)

接下来，创建并运行一个`HyperparameterTuningJob`。

需要注意的一些参数是：

* `max_trial_count`: 设置服务运行试验的最大次数上限。建议的做法是从较小数量的试验开始，并了解你选择的超参数在扩展之前的影响程度。

* `parallel_trial_count`: 如果您使用并行试验，服务将为多个训练处理集群提供资源。在创建作业时指定的工作池规范将用于每个单独的训练集群。增加并行试验的数量可以减少超参数调整作业的运行时间；但是，这可能会降低作业的效果。这是因为默认的调整策略使用先前试验的结果来指导后续试验中值的分配。

* `search_algorithm`: 可用的搜索算法有网格、随机或默认(None)。默认选项应用贝叶斯优化来搜索可能的超参数值空间，是推荐的算法。

In [ ]:
# Create and run HyperparameterTuningJob

hp_job = aiplatform.HyperparameterTuningJob(
    display_name=JOB_NAME,
    custom_job=my_custom_job,
    metric_spec=metric_spec,
    parameter_spec=parameter_spec,
    max_trial_count=15,
    parallel_trial_count=3,
    project=PROJECT_ID,
    search_algorithm=None,
)

hp_job.run()

点击输出中生成的链接以在云控制台中查看您的运行情况。当作业完成时，您将看到调试试验的结果。

将以下英文文本翻译为中文: ![console_ui_results](tuning_results.png)

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源。

In [ ]:
# Set this to true only if you'd like to delete your bucket
delete_bucket = False

hp_job.delete()

!gcloud container images delete $IMAGE_URI --force-delete-tags --quiet

if delete_bucket:
    ! gsutil rm -r $BUCKET_URI